In [73]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [ ]:
client = MongoClient('localhost', 27017)

## Cases and deaths
- by day and county
- prepared by Aleksanda

### CSV to Parquet conversion

In [ ]:
# pd.read_csv(
#     "../data/cases_daily_csv.csv", 
#     usecols=["date", "fips", "cases", "deaths"]
# ).to_parquet(
#     "../data/cases_dailt.parquet",
#     engine="pyarrow", 
#     compression="brotli"
# )

In [ ]:
df_cases = pd.read_parquet("../data/cases_daily.parquet")
df_cases.fips = df_cases.fips.astype(str).str.zfill(5)
df_cases.head()

In [ ]:
data_dict_by_fips = [{"fips": fips, "data": df.drop(columns=["fips"]).to_dict("records")} for fips, df in df_cases.groupby("fips")]

In [ ]:
client["MDAProjectDatabase"]["daily_covid_cases"].insert_many(data_dict_by_fips)

## Socio-economic data

In [ ]:
df_socio_economic = pd.read_csv("../data/non_epidemic_data.csv")
print(df_socio_economic.shape)
df_socio_economic.head()

In [ ]:
fipses_cases = pd.read_csv("../data/county_daily_csv.csv", usecols=["fips"]).fips.values

df_socio_economic = df_socio_economic.loc[df_socio_economic.fips.isin(fipses_cases), :]
df_socio_economic.shape

In [ ]:
client["MDAProjectDatabase"]["socio_economic_data"].insert_many(df_socio_economic.drop(columns=["area"]).to_dict("records"))

## FIPS dictionary

In [ ]:
df_fips = pd.read_csv("../data/fips.csv")

df_fips_states = df_fips.loc[df_fips.fips.values % 1000 == 0, :].copy()
df_fips_states.loc[:, "fips_state"] = [f"{int(fips / 1000):02d}" for fips in df_fips_states.fips]
df_fips_states = df_fips_states.drop(columns=["fips"]).rename(columns={"area": "state_name"})

df_fips = df_fips.loc[df_fips.fips.values % 1000 != 0, :]
df_fips.loc[:, "fips_state"] = [f"{fips:05d}"[:2] for fips in df_fips.fips]

df_fips = pd.merge(
    df_fips,
    df_fips_states,
    on="fips_state"
).drop(columns=["fips_state"])

df_fips.fips = df_fips.fips.astype(str).str.zfill(5)

df_fips.head()

In [ ]:
df_fips.head().to_dict("records")

In [ ]:
client["MDAProjectDatabase"]["fips_codes"].insert_many(df_fips.to_dict("records"))